In [1]:
from os import listdir
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import numpy as np
from tqdm import tqdm
import wikipedia
from nltk.metrics.distance import edit_distance
import re

In [2]:
MISSING = ""

# Scrapping chart

In [3]:
years = [str(y) for y in np.arange(2004, 2023, 1)]
weeks = [int(v) for v in np.linspace(1, 52, 12)]
mois = ["Janvier", "Fevrier", "Mars", "Avril", "Mai", "Juin", "Juillet", "Aout", "Septembre", "Octobre", "Novembre", "Decembre"]

In [4]:
dfs_chart = {y : {} for y in years}
data = []

for y in tqdm(years):
    for k, w in enumerate(weeks):
        url = f"https://acharts.co/france_singles_top_100/{y}/{w}"
        rq = get(url)
        m = mois[k]
        if rq.ok:
            soup = BeautifulSoup(rq.text)

            chart = soup.find("table", {"id" : "ChartTable"})
            trs = chart.findAll("tr")[1:]

            for tr in trs:
                music_name = tr.find("span", {"itemprop" : "name"}).text.lower()
                rank = tr.find("span", {"itemprop" : "position"}).text.lower()
                artist_name = tr.find("span", {"itemprop" : "byArtist"}).text[2:-1].lower()
                data.append([m, y, str(rank), str(artist_name), str(music_name)])

charts = pd.DataFrame(data, columns=["Mois", "Annee", "Rank", "Artist", "Music"])

100%|██████████| 19/19 [02:33<00:00,  8.10s/it]


In [5]:
#Export
charts.to_csv("../etapes/1 - scrapping chart/charts.csv", index=False, encoding="utf-8-sig")

# Cleanning chart

In [46]:
names = [
    ("Djadja", "Djadja et Dinaz"),
    ("Lorie", "Lorie Pester"),
    ("-m-", "Matthieu Chedid"),
    ("Priscilla", "Priscilla Betti"),
    ("I Am", "IAM"),
    ("Sofiane", "Sofiane Zermani"),
    ("Justice", "Justice (groupe)"),
    ("Soma Riba", "Collectif Métissé"),
    ("Fresh", "Fresh la Peufra"),
    ("Rosalia", "Rosalía"),
    ("Italo Brothers", "ItaloBrothers"),
    ("Far\*east Movement", "Far East Movement"),
    ("Odyssey", "Odyssey (groupe)"),
    ("1789", "1789 : Les Amants de la Bastille"),
    ("Clemence", "Clémence Saint-Preux"),
    ("Rose", "Rose (chanteuse)"),
    ("Laeti", "Laetitia Kerfa"),
    ("La Troupe", "Mozart, l'opéra rock"),
    ("Victoria", "Victoria Sio"),
    ("Christine And The Queens", "Redcar (artiste)"),
    ("Earth and Wind And Fire", "Earth, Wind And Fire"),
    ("Jean Roch", "Jean-Roch"),
    ("Keen V", "Keen'V"),
    ("Dinor", "Dinor RDT"),
    ("Koba La D", "Koba LaD"),
    ("scotts", "Travis Scott"),
    ("louane emera", "louane"),
    ("maitre gims", "gims"),
    ("p!nk", "pink"),
    ("Eva", "Eva Queen"),
    ("dj tiësto", "tiesto"),
    ("Karol", "Karol G"),
    ("Khaled", "DJ Khaled"),
    ("Black Eyed Peas", "The Black Eyed Peas"),
    ("L.E.J", "LEJ"),
    ("Disiz la peste", "Disiz"),
    ("k'maro", "k. maro"),
    ("shin sekaï", "The Shin Sekaï"),
    ("the niro", "niro"),
    ("r.i.o.", "rio"),
    ("zayn", "zayn malik"),
    ("dimitri vegas", "dimitri vegas & like mike"),
    ("cauet", "Sébastien Cauet"),
    ("do", "the do"),
    ("les filles", "Aurélie Konaté")
]

replace_words = [
    ("\x9c", "oe"),
    ("œ", "oe"),
    ("$", "S"),
]

charts["Artist"] = charts["Artist"].str.split(" x ", regex=False).str[0]
charts["Artist"] = charts["Artist"].str.split(" - ", regex=False).str[-1]
charts["Artist"] = charts["Artist"].str.split(" \+ ", regex=False).str[0]

charts["Artist"] = charts["Artist"].apply(lambda x: "star academy" if "star academy" in x else x)

for n1, n2 in names:
    charts["Artist"] = charts["Artist"].apply(lambda x: n2.lower() if x.lower() == n1.lower() else x.lower())

for n1, n2 in replace_words:
    charts["Artist"] = charts["Artist"].str.replace(n1.lower(), n2.lower(), regex=False)


In [47]:
#Export
charts.to_csv("../etapes/2 - cleanning chart/charts.csv", index=False, encoding="utf-8-sig")

# Scrapping artist data

In [50]:
def find_title_in_wikipedia(title, pourcentage=0.3):
    words = ["(chanteur)", "(chanteuse)", "(groupe)", "(rappeur)", "(rappeuse)", "(musicien)", "(chanteur français)", "(france)", "(producteur)", "(artiste)", "(groupe de musique)"]

    wikipedia.set_lang("fr")
    results = wikipedia.search(title, results=10)
    distance = []
    if len(results) > 0:
        for element in results:
            if any((w in element.lower()) and (edit_distance(element.lower().split(" (")[0].strip(), title.lower().strip())/len(title) < pourcentage) for w in words):
                return element

            distance.append(edit_distance(title.lower().strip(), element.lower().strip()))

        return results[np.argmin(distance)] if min(distance)/len(title) < pourcentage else MISSING

In [51]:
def wiki_birth(title):
    cols = ["Naissance", "Pays d'origine", "Origine", "Nationalité", "Pays", "Summary"]
    nats = ["franco", "français", "belge", "canadien", "libanais", "réunionnais"]
    dic = {w : MISSING for w in cols}

    if title == MISSING:
        return dic

    url = f"https://fr.wikipedia.org/wiki/{title}"
    rq = get(url)

    if not rq.ok:
        return dic
    
    soup = BeautifulSoup(rq.text)
    tables = soup.findAll("table")

    for table in tables:
        trs = table.findAll("tr")

        for tr in trs:
            th = tr.find("th")

            if th is not None:
                for w in cols:
                    if w in th.text:
                        td = tr.find("td")
                        if td is not None:
                            dic[w] = td.text.strip().lower()

    wikipedia.set_lang("fr")
    try:
        summary = wikipedia.summary(title, sentences=1)
        dic["Summary"] = summary.lower().strip()

        if dic["Nationalité"] == MISSING:
            for w in nats:
                if w in summary:
                    dic["Nationalité"] = w.lower().strip()
                    return dic
    except:
        pass

    return dic

In [52]:
#Récupére tous les artistes uniques
artist = pd.DataFrame(charts["Artist"].unique(), columns=["Artist"])

In [53]:
artist["Artist_wiki"] = artist["Artist"].apply(lambda x : find_title_in_wikipedia(x, 0.2)) #Trouve les pages wikipedia de chaque artistes

In [54]:
birth_dic = artist["Artist_wiki"].apply(wiki_birth) #Cherche les infos de naissance sur les pages wikipedia

c:\Users\quent\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\quent\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [55]:
#Transforme les infos trouver sur wikipédia en dataframe
dfs_birth = []
for dic in birth_dic:
    dfs_birth.append(pd.DataFrame(dic, index=[0]))
birth = pd.concat(dfs_birth, ignore_index=True)

In [56]:
#Merge les infos de naissance avec les infos des artistes
artist = artist.merge(birth, left_index=True, right_index=True)
artist = artist.set_index("Artist")

In [57]:
#On ajoute manuellement les informations pour les artistes importants qui n'ont pas été trouvé sur wikipedia
manuel_names = [
    ("sound of legend", "MANUEL", MISSING, MISSING, MISSING, "français", MISSING, MISSING),
    ("al. hy", "MANUEL", "15 novembre 1993", MISSING, MISSING, "français", MISSING, "15 novembre 1993 (Âge: 29 ans), Saint-Amand-les-Eaux"),
    ("glk", "MANUEL", MISSING, MISSING, MISSING, "français", MISSING, "Originaire de Bobigny, en Seine-Saint-Denis"),
    ("paul glaeser", "MANUEL", "1963", MISSING, MISSING, "français", MISSING, MISSING),
    ("bolémvn", "MANUEL", "29 novembre 1996", MISSING, MISSING, "français", MISSING, "Bryan Mounkala (né le 29 novembre 1996) mieux connu sous le nom de Bolémvn est un rappeur français d'Évry, Essonne, Île-de-France, France."),
    ("paul glaeser", "MANUEL", "1963", MISSING, MISSING, "français", MISSING, MISSING),
    ("1pliké140", "MANUEL", "1963", MISSING, MISSING, "français", MISSING, "1PLIKÉ140 est un jeune rappeur français originaire de Clamart (92) en banlieue parisienne."),
    ("jérôme collet", "MANUEL", MISSING, MISSING, MISSING, "français", MISSING, MISSING),
    ("funnybear", "MANUEL", MISSING, MISSING, MISSING, "français", MISSING, MISSING),
    ("tom snare", "MANUEL", MISSING, MISSING, MISSING, "français", MISSING, "Xavier Decanter, mieux connu sous son nom de scène Tom Snare, est un DJ et producteur de disques français originaire de Dunkerque."),
    ("funnybear", "MANUEL", "22 janvier 1991", MISSING, MISSING, "français", MISSING, "22 janvier 1991 (Âge: 31 ans), Hyères"),
    ("landy", "MANUEL", "2000", MISSING, MISSING, "français", MISSING, "Dylan Sylla Gahoussou, « Landy » de son nom de scène, est né dans le XIXe arrondissement de Paris"),
    ("dry", "MANUEL", "19 novembre 1977", MISSING, MISSING, "français", MISSING, "Landry Delica a grandi à Sevran en Seine-Saint-Denis"),
    ("sasso", "MANUEL", MISSING, MISSING, MISSING, "français", MISSING, "Né à Vénissieux d'un père togolais et d'une mère marocaine"),
    ("neïman", "MANUEL", MISSING, MISSING, MISSING, "guyanais", MISSING, "NEÏMAN est un chanteur et toaster français de dancehall et de reggae-soul né RAY NEÏMAN en Guyane."),
    ("isk", "MANUEL", "20 mars 2003", MISSING, MISSING, "français", MISSING, "ISK, de son vrain nom Kais Ben Baccar, est un rappeur français d'origine tunisienne, né au Canada. Habitant La Ferté-sous-Jouarre (77), il est membre du label Bendo 11 Records sous la division rap GrandLine."),
    ("yaro", "MANUEL", "1996", MISSING, MISSING, "français", MISSING, "Yaro, anciennement Sirsy, est un rappeur français originaire de la ville de Yerres dans l'Essonne.")
]

manuel_artist = pd.DataFrame(manuel_names, columns=["Artist", "Artist_wiki", "Naissance", "Pays d'origine", "Origine", "Nationalité", "Pays", "Summary"]).set_index("Artist")
artist.drop(index=manuel_artist.index, inplace=True)
artist = pd.concat([artist, manuel_artist])

In [58]:
#Export
artist.to_csv("../etapes/3 - scrapping artist data/artist.csv", encoding="utf-8-sig")

# Cleanning Wikipédia results

## Nationalité/Pays/Origine

In [59]:
gentille_df = pd.read_csv("../monde/gentille.csv")
gentille = gentille_df["gentille"].tolist()
pays = gentille_df["pays"].tolist()

In [60]:
def cleanning(data, replace_words):
    if data == MISSING:
        return MISSING
    else:
        distances = []
        for r in replace_words:
            distances.append(edit_distance(data.lower(), r.lower()))
        return replace_words[np.argmin(distances)]

In [61]:
artist["Nationalité"] = artist["Nationalité"].apply(lambda x : cleanning(x, gentille))
artist["Origine"] = artist["Origine"].apply(lambda x : cleanning(x, gentille))

In [62]:
artist["Pays"] = artist["Pays"].apply(lambda x : cleanning(x, pays))
artist["Pays d'origine"] = artist["Pays d'origine"].apply(lambda x : cleanning(x, pays))

In [63]:
#Ajout de la colonne "No data" pour voir ceux sur qui on n'a pas de données
artist["No data"] = (
    (artist["Naissance"] == MISSING) &
    (artist["Pays d'origine"] == MISSING) &
    (artist["Origine"] == MISSING) &
    (artist["Nationalité"] == MISSING) &
    (artist["Pays"] == MISSING) &
    (artist["Summary"] == MISSING)
)

In [64]:
artist = artist[~artist["No data"]] #On ne garde que les artistes pour lesquels on a des données

In [65]:
def get_nationality(row, words):
    for r in words:
        regex = r"([\d)()\] ]|^)"+ r.lower() + r"([.,\[) ]|$)"

        if not re.search(regex, row["Summary"].lower()) is None:
            return r.lower()
    return MISSING

In [66]:
#On utilise le summary pour récupérer la nationalité
artist.loc[artist["Nationalité"] == MISSING, "Nationalité"] = artist[artist["Nationalité"] == MISSING].apply(lambda x: get_nationality(x, artist["Nationalité"].unique()), axis=1)

In [67]:
def get_pays(row, words):
    for r in words:
        regex = r"([\d)()\], ]|^)"+ r.lower() + r"([.,\[) ]|$)"

        if not re.search(regex, row["Naissance"].lower()) is None:
            return r.lower()
    return MISSING

In [68]:
# On récupére une liste de pays
gentille_df = pd.read_csv("../monde/gentille.csv")
pays = gentille_df["pays"].tolist()

In [69]:
#On utilise le summary pour récupérer la Pays
artist.loc[artist["Pays"] == MISSING, "Pays"] = artist[artist["Pays"] == MISSING].apply(lambda x: get_pays(x, pays), axis=1)

In [70]:
#On récupère la nationalité à partir du pays
gentille = pd.DataFrame.to_dict(gentille_df.set_index("pays"), orient="dict").get("gentille")

for k, v in gentille.items():
    artist.loc[(artist["Nationalité"] == MISSING) & (artist["Pays"] == k), "Nationalité"] = v
    artist.loc[(artist["Nationalité"] == MISSING) & (artist["Pays d'origine"] == k), "Nationalité"] = v

In [71]:
# Les origines sont transposées avec sur la nationalité
artist.loc[(artist["Nationalité"] == MISSING) & (artist["Origine"] != MISSING), "Nationalité"] = artist.loc[(artist["Nationalité"] == MISSING) & (artist["Origine"] != MISSING), "Origine"]

In [72]:
#Export
artist.to_csv("../etapes/4 - cleanning wikipedia results/nationalité_pays/artist.csv", encoding="utf-8-sig")

## Commune/Departement/Region

In [73]:
#Load
artist = pd.read_csv("../etapes/4 - cleanning wikipedia results/nationalité_pays/artist.csv")
artist.fillna(MISSING, inplace=True)

In [74]:
region = pd.read_csv("../france/departements-france.csv")
departement = pd.read_csv("../france/departements-france.csv")
commune = pd.read_csv("../france/communes-departement-region.csv")
nb_habitant = pd.read_csv("../france/nb_habitant.csv")

In [75]:
#On supprime les colonnes inutiles
commune = commune.drop(columns = ["code_commune_INSEE", "nom_commune_postal", "code_postal", "libelle_acheminement", "ligne_5", "latitude", "longitude", "code_commune", "article", "code_departement", "code_region", "nom_commune"])

#On rename la colonne nom_commune_complet en nom_commune
commune = commune.rename(columns={"nom_commune_complet": "nom_commune"})

#On enleve les arrondisement des villes
commune.loc[commune["nom_commune"].str.contains(r"[A-Za-z]* [0-9]{2}"), "nom_commune"] = commune.loc[commune["nom_commune"].str.contains(r"[A-Za-z]* [0-9]{2}"), "nom_commune"].str[:-3]

#On ajoute le nombre d'habitant au commune
commune = commune.merge(nb_habitant[["Ville", "nb_habitant"]], left_on="nom_commune", right_on="Ville", how="inner").drop(columns=["Ville"]).dropna().drop_duplicates()

#On transforme le nb d'habitant en int
commune["nb_habitant"] = commune["nb_habitant"].str.replace(" ", "")
commune["nb_habitant"] = commune["nb_habitant"].astype("int")

#On transforme tout en lower
for col in commune.select_dtypes("object").columns:
    commune[col] = commune[col].str.lower()

In [76]:
def get_localisation(row, localisation):
    for r in localisation:
        regex = r"([\d)()\] ]|^)"+ r.lower() + r"([.,\[) ]|$)"

        if (not re.search(regex, row["Summary"].lower()) is None) or (not re.search(regex, row["Naissance"].lower()) is None):
            return r.lower()
    return MISSING

In [77]:
artist["Region"] = artist.apply(lambda x: get_localisation(x, region["nom_region"].unique()), axis=1)

In [78]:
artist["Commune"] = artist.apply(lambda x: get_localisation(x, commune[commune["nb_habitant"] > 2000]["nom_commune"].unique()), axis=1)

In [79]:
artist["Departement"] = artist.apply(lambda x: get_localisation(x, departement["nom_departement"].unique()), axis=1)

In [80]:
artist["No localisation"] = (
    (artist["Region"] == MISSING) &
    (artist["Commune"] == MISSING) &
    (artist["Departement"] == MISSING)
)

In [81]:
# Les artistes pour qui ont a trouvé une commune/departement/region on leur donne la nationalité française
artist.loc[(artist["Nationalité"] == MISSING) & (~artist["No localisation"]), "Nationalité"] = "français"

In [82]:
def commune_to_departement(x : str):
    value = commune.loc[commune["nom_commune"] == x, "nom_departement"].values
    if len(value) > 0:
        return value[0]
    else:
        return MISSING

In [83]:
#On ajoute les departements pour les communes
artist.loc[(artist["Commune"] != MISSING) & (artist["Departement"] == MISSING), "Departement"] = artist.loc[(artist["Commune"] != MISSING) & (artist["Departement"] == MISSING), "Commune"].apply(commune_to_departement)

In [84]:
def departement_to_region(x : str):
    value = commune.loc[commune["nom_departement"] == x, "nom_region"].values
    if len(value) > 0:
        return value[0]
    else:
        return MISSING

In [85]:
#On ajoute les regions pour les departements
artist.loc[(artist["Departement"] != MISSING) & (artist["Region"] == MISSING), "Region"] = artist.loc[(artist["Departement"] != MISSING) & (artist["Region"] == MISSING), "Departement"].apply(departement_to_region)

In [86]:
#Export
artist.to_csv("../etapes/4 - cleanning wikipedia results/departement_region/artist.csv", encoding="utf-8-sig")